<a href="https://colab.research.google.com/github/Adilmar/Michelzinho-Gemini/blob/main/gemini_alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importar as bibliotecas**

In [ ]:
!pip install google-generativeai
from pathlib import Path
import hashlib
import google.generativeai as genai
from google.colab import userdata

**Declarar variavel com a chave da api**

In [ ]:
API_KEY = userdata.get('GEMINI')

In [ ]:
def gemini(api_key, text, objeto):
  genai.configure(api_key=api_key)

  # Set up the model
  generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
    "stop_sequences": [],
  }

  safety_settings = [
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
  ]

  #aqui vai a instrucao para o seu chatbot
  system_instruction = "Você é um chatbot chamado Michelzinho para crianças com autismo, converse no mesmo tom. Você foi desenvolvido por Adilmar Coelho Dantas Doutor em Ciência da Computação  pela universidade federal de Uberlândia no ano de 2021. Você é capaz de analisar a emoção também pela foto da criança. Seu site é https://michelzinho.pushsistemas.com.br, você sabe falar sobre as emoções para explicar como espessar e interpretar."

  model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",generation_config=generation_config,system_instruction=system_instruction,safety_settings=safety_settings)

  uploaded_files = []
  def upload_if_needed(pathname: str) -> list[str]:
    path = Path(pathname)
    hash_id = hashlib.sha256(path.read_bytes()).hexdigest()
    try:
      existing_file = genai.get_file(name=hash_id)
      return [existing_file.uri]
    except:
      pass
    uploaded_files.append(genai.upload_file(path=path, display_name=hash_id))
    return [uploaded_files[-1].uri]

  convo = model.start_chat(history=objeto)

  convo.send_message(text)

  for uploaded_file in uploaded_files:
    genai.delete_file(name=uploaded_file.name)

  resposta = str(convo.last.text)

  #gerando response em json
  response = {"StatusCode":200, "resposta": resposta}

  return response

In [ ]:
historico = []

mensagem = ""

#condicao de repeticao para a conversa
while mensagem != "#sair":
  chat = str(input("👤 Digite sua mensagem...."))
  mensagem = chat
  historico.append({"parts":[chat],"role":"user"})
  response = gemini(API_KEY,chat,historico)
  resposta = str(response["resposta"])
  resposta = f"🐶 {resposta}"
  print(resposta)
  historico.append({"parts":[resposta],"role":"model"})
